In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

In [2]:
years = [2019,2020]
file_dirs = []
for year in years:
    for filename in os.listdir(f'../data/{year}/'):
        file_dirs.append(f'../data/{year}/{filename}')
        
countries_df = [pd.read_csv(file_dir) for file_dir in file_dirs]
df = pd.concat(countries_df, ignore_index=True)

df['Period'] = pd.to_datetime(df['Period'], format='%Y%m')
cols = [
    'Period',
    'Reporter',
    'Reporter Code',
    'Trade Value (US$)',
    'Trade Flow Code',
    'Mode of Transport Code',
    'Commodity Code',
    'Commodity',
    'Customs Proc. Code',
    'Partner',
    '2nd Partner'
]
df = df[cols]

In [3]:
tot_volume_filt = (
    (df['Trade Flow Code'].isin([2, np.nan])) &  # Export only
    (df['Mode of Transport Code'].isin([0, np.nan])) &  # all modes of transport
    (df['Customs Proc. Code'].isin(['C00', np.nan])) &  # all customs protocol
    (df['Partner'] == 'World') &
    (df['2nd Partner'].isin(['World', np.nan]))
)
df = df[tot_volume_filt]
df = df.sort_values(by=['Period', 'Reporter'])

In [4]:
def split_df(df, period, pre_covid_start):
    '''
    args:
        df (pd.df): original df with data on all available countries
        period (int): duration of pre- and post-covid timeline, in months
        pre_covid_start (datetime): begining of the pre covid period
    
    returns:
        pre_COVID_df (pd.df): pre covid df for countries that have complete data for pre- & post-covid period 
        post_COVID_df (pd.df): post covid df for countries that have complete data for pre- & post-covid period 
    '''
    pre_covid_periods = pd.date_range(pre_covid_start, periods=(period), freq='MS', closed=None)
    post_covid_periods = pd.date_range(pre_covid_periods[-1], periods=(period+1), freq='MS', closed='right')
    expected_dates = np.append(pre_covid_periods, post_covid_periods)
    
    useable_codes = [] 
    for code in df['Reporter Code'].unique():
        available_dates = df[df['Reporter Code']==code]['Period'].unique()
        if np.setdiff1d(expected_dates, available_dates).size == 0: 
            useable_codes.append(code)
            
    useable_df = df[df['Reporter Code'].isin(useable_codes)]
    pre_COVID_df = useable_df[useable_df['Period'].isin(pre_covid_periods)]
    post_COVID_df = useable_df[useable_df['Period'].isin(post_covid_periods)]
    
    return pre_COVID_df, post_COVID_df

In [5]:
period = 5 
pre_covid_start = datetime.datetime(2019,7,1)
pre_COVID_df, post_COVID_df = split_df(df, period, pre_covid_start)

post_COVID_tot_vol = post_COVID_df.groupby(['Commodity Code'])['Trade Value (US$)'].sum()
pre_COVID_tot_vol = pre_COVID_df.groupby(['Commodity Code'])['Trade Value (US$)'].sum()

In [6]:
HS2_commodities = pd.read_csv('../data/HS2 key/HS2 code.csv', index_col=0)
HS2_commodities[f'{period}Ms Growth (US$)'] = post_COVID_tot_vol - pre_COVID_tot_vol
HS2_commodities[f'{period}Ms Growth (%)'] = (post_COVID_tot_vol - pre_COVID_tot_vol) / pre_COVID_tot_vol

HS2_commodities[HS2_commodities[f'{period}Ms Growth (%)']<0]

Commodity  5Ms Growth (US$)  \
HS2 Code                                                                        
1                                              LIVE ANIMALS     -1.880662e+08   
3         FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT...     -4.307226e+09   
11        PRODUCTS OF THE MILLING INDUSTRY; MALT; STARCH...     -1.246810e+08   
12        OIL SEEDS AND OLEAGINOUS FRUITS; MISCELLANEOUS...     -1.234319e+09   
14        VEGETABLE PLAITING MATERIALS; VEGETABLE PRODUC...     -1.086418e+07   
...                                                     ...               ...   
94        FURNITURE; BEDDING, MATTRESSES, MATTRESS SUPPO...     -8.659750e+09   
95        TOYS, GAMES AND SPORTS REQUISITES; PARTS AND A...     -6.949882e+09   
96                      MISCELLANEOUS MANUFACTURED ARTICLES     -7.630799e+08   
97            WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES     -4.358548e+09   
99             COMMODITIES NOT SPECIFIED ACCORDING TO KIND      -8.175649e+09   

          5Ms Growth (%)  
HS2 Code                  
1              -0.033180  
3              -0.175205  
11             -0.030007  
12             -0.061238  
14             -0.095708  
...                  ...  
94             -0.187765  
95             -0.330825  
96             -0.073491  
97             -0.321258  
99             -0.098578  

[79 rows x 3 columns]